## Read me
#### Always run until '[Generating label feature txt file](#Iwork)' then choose what file generator is needed

In [2]:
import cntk as C
from scipy.misc.pilutil import imread
from matplotlib import pyplot as plt
import numpy as np
from os.path import join, dirname, exists, isfile
from os import listdir, rmdir, makedirs
import sys
from urllib.request import urlretrieve
import struct
import xml.etree.cElementTree as et
import xml.dom.minidom
%matplotlib inline

In [3]:
# WOOOOOOOOOOOOOOOO (You get the D later)
imgSize = 200
numFeatures = imgSize * imgSize * 3
root = 'Wood_Dataset'
# map_path = '~/Desktop/University/Imperial_College/Project/code/Wood_Dataset'

## Generating label feature txt file

In [6]:
def loadData_train(root):
    print('Loading Training data')
    path_features = join(root, 'Train')
    images = np.zeros(shape=[5708,(200 * 200 * 3) + 1], dtype=np.int32)
    num_images = 0
    for i in range(1,13):
        path = join(path_features,str(i))
        print(path)
        files = listdir(path)
        for count in range(0, len(files)):
            img_path = join(path, files[count])
            img = imread(img_path)
            images[count + num_images,:-1] = img.flatten()
            images[count + num_images, -1] = i
        print('Accessing images with label : {}'.format(str(i)))
        num_images = num_images + len(files)
    return images

In [7]:
#save data
# Save the data files into a format compatible with CNTK text reader
def savetxt(filename, ndarray):
    dir = dirname(filename)

    if not exists(dir):
        makedirs(dir)
    
    if not isfile(filename):
        print("Saving", filename )
        with open(filename, 'w') as f:
            labels = list(map(' '.join, np.eye(12, dtype=np.uint).astype(str)))
            for row in ndarray:
                row_str = row.astype(str)
                label_str = labels[row[-1] - 1] #needs +1 as index start at 0
                feature_str = ' '.join(row_str[:-1])
                f.write('|labels {} |features {}\n'.format(label_str, feature_str))
    else:
        print("File already exists", filename)

In [8]:
# train and test done seperatly to avoid memory error
data_train = loadData_train(root)

Loading Training data
Wood_Dataset/Train/1
Accessing images with label : 1
Wood_Dataset/Train/2
Accessing images with label : 2
Wood_Dataset/Train/3
Accessing images with label : 3
Wood_Dataset/Train/4
Accessing images with label : 4
Wood_Dataset/Train/5
Accessing images with label : 5
Wood_Dataset/Train/6
Accessing images with label : 6
Wood_Dataset/Train/7
Accessing images with label : 7
Wood_Dataset/Train/8
Accessing images with label : 8
Wood_Dataset/Train/9
Accessing images with label : 9
Wood_Dataset/Train/10
Accessing images with label : 10
Wood_Dataset/Train/11
Accessing images with label : 11
Wood_Dataset/Train/12
Accessing images with label : 12


In [37]:
sample_num = 4000
data_show = data_train[sample_num,:-1].reshape(200,200,3)
print(data_show.shape)
plt.imshow(data_show)
plt.show()
print("Image Label: {}".format(data_train[sample_num,-1]))

NameError: name 'data_train' is not defined

In [ ]:
data_dir = join('data','wood')
print('Writing train text file...')
savetxt(join(data_dir, 'Train-Wood-Species-cntk.txt'), data_train)


In [ ]:
def loadData_test(root):
    print('Loading Test Data')
    path_features = join(root, 'Test')
    files = listdir(path_features)
    images = np.zeros(shape=[len(files),(200 * 200 * 3) + 1], dtype=np.int32)
    for count in range(0, len(files)):
        lbl = (files[count].split('_'))[1].split('.')[0]
        img_path = join(path_features, files[count])
        img = imread(img_path)
        images[count,:-1] = img.flatten()
        images[count , -1] = lbl
    return images

In [ ]:
# train and test done seperatly to avoid memory error
data_test = loadData_test(root)

In [ ]:
# saving test data
print('Writing test text file ...')
savetxt(join(data_dir, 'Test-Wood-Species-cntk.txt'), data_test)

## Generating label image map

In [14]:
def createMap(Train, filename):
    print('Getting {} data'.format(('train' if Train else 'test')))
    dir = dirname(filename)
    with open(filename, 'w') as f:
        if not exists(dir):
            makedirs(dir)
        if Train:
            path = join(root, 'Train')
            for i in range(1, 13):
                image_path = join(path, str(i))
                print('Accessing {}'.format(image_path))
                file_list = listdir(image_path)
                for file in file_list:
                    f.write("%s\t%d\n" % (join(image_path, file), i - 1))
        else:
            image_path = join(root, 'Test')
            print('Accessing {}'.format(image_path))
            file_list = listdir(image_path)
            for file in file_list:
                    lbl = (file.split('_'))[1].split('.')[0]
                    f.write("%s\t%d\n" % (join(image_path, file), int(lbl)-1))

In [15]:
#Training map generation
createMap(True, 'data/wood/Train-Wood-Species-Map.txt')
#Testing map generation
createMap(False, 'data/wood/Test-Wood-Species-Map.txt')

Getting train data
Accessing Wood_Dataset/Train/1
Accessing Wood_Dataset/Train/2
Accessing Wood_Dataset/Train/3
Accessing Wood_Dataset/Train/4
Accessing Wood_Dataset/Train/5
Accessing Wood_Dataset/Train/6
Accessing Wood_Dataset/Train/7
Accessing Wood_Dataset/Train/8
Accessing Wood_Dataset/Train/9
Accessing Wood_Dataset/Train/10
Accessing Wood_Dataset/Train/11
Accessing Wood_Dataset/Train/12
Getting test data
Accessing Wood_Dataset/Test


## Generating mean XML mean file

In [5]:
def saveMean(fname, data):
    print('Saving XML file')
    root = et.Element('opencv_storage')
    et.SubElement(root, 'Channel').text = '3'
    et.SubElement(root, 'Row').text = str(imgSize)
    et.SubElement(root, 'Col').text = str(imgSize)
    meanImg = et.SubElement(root, 'MeanImg', type_id='opencv-matrix')
    et.SubElement(meanImg, 'rows').text = '1'
    et.SubElement(meanImg, 'cols').text = str(imgSize * imgSize * 3)
    et.SubElement(meanImg, 'dt').text = 'f'
    et.SubElement(meanImg, 'data').text = ' '.join(['%e\n' % n for n in np.reshape(data, (imgSize * imgSize * 3))])

    tree = et.ElementTree(root)
    tree.write(fname)
    x = xml.dom.minidom.parse(fname)
    with open(fname, 'w') as f:
        f.write(x.toprettyxml(indent = '  '))
    print('Done!')    
        
def saveTrainMean():
    print('Loading Training data')
    path_features = join(root, 'Train')
    images = np.zeros(shape=[5708,(200 * 200 * 3)], dtype=np.int32)
    num_images = 0
    for i in range(1,13):
        path = join(path_features,str(i))
        print(path)
        files = listdir(path)
        for count in range(0, len(files)):
            img_path = join(path, files[count])
            img = imread(img_path)
            images[count + num_images,:] = img.flatten()
        print('Accessing images with label : {}'.format(str(i)))
        num_images = num_images + len(files)
    images = (np.sum(images, axis=0))/5708
    saveMean('data/wood/Wood-Data-Mean.xml',images)
    
    

In [6]:
# Run mean calculator and xml saver
saveTrainMean()

Loading Training data
Wood_Dataset/Train/1
Accessing images with label : 1
Wood_Dataset/Train/2
Accessing images with label : 2
Wood_Dataset/Train/3
Accessing images with label : 3
Wood_Dataset/Train/4
Accessing images with label : 4
Wood_Dataset/Train/5
Accessing images with label : 5
Wood_Dataset/Train/6
Accessing images with label : 6
Wood_Dataset/Train/7
Accessing images with label : 7
Wood_Dataset/Train/8
Accessing images with label : 8
Wood_Dataset/Train/9
Accessing images with label : 9
Wood_Dataset/Train/10
Accessing images with label : 10
Wood_Dataset/Train/11
Accessing images with label : 11
Wood_Dataset/Train/12
Accessing images with label : 12
Saving XML file
Done!
